In [1]:
%run "Common setup.ipynb"

Because SALib concentrates on GSA methods, we implement our own local OAT methods here.

In [2]:
# problem specification for morris analysis
oat_spec = {
  'num_vars': 4,
  'names': ['x1', 'x2', 'x3', 'x4'],
  'bounds': [[-np.pi, np.pi]]*4,
  'nominal': [1.0,] * 4
}

In [3]:
cols = np.arange(5)
cols[cols != 2]

array([0, 1, 3, 4])

In [4]:
def oat_extremity_sample(problem):
    """Create extremity samples for a 'traditional' OAT analysis.
    
    Here we assume outputs will be affected at the extreme ends of the nominal bounds.
    
    Parameters
    ==========
    * problem : dict, the problem definition following SALib specs        
    """
    bounds = problem.get('bounds')
    nominal = problem.get('nominal')
    num_vars = problem.get('num_vars')
    
    # nominal = np.array(nominal)
    experiment = [nominal[:]]
    
    # Samples represent runs with values at the extremes of each parameter boundary
    for i in range(num_vars):
        min_val = bounds[i][0]
        max_val = bounds[i][1]
        
        after = i+1
        if i > 0:
            experiment.append(nominal[0:i] + [min_val] + nominal[after:])  # min
            experiment.append(nominal[0:i] + [max_val] + nominal[after:])  # max
        else:
            experiment.append([min_val] + nominal[after:])  # min
            experiment.append([max_val] + nominal[after:])  # max
            
    
    return np.array(experiment)
# End oat_extremity_sample()


def traditional_oat_sample(problem, N):
    """Create samples for a 'traditional' OAT analysis.
    
    Parameters
    ==========
    * problem : dict, the problem definition following SALib specs        
    """
    bounds = problem.get('bounds')
    nominal = problem.get('nominal')
    num_factors = int(problem.get('num_vars'))
    
    samples_per_param = N / num_factors
    
    if samples_per_param % 1 != 0:
        raise ValueError("Require equal number of samples per parameter. \
        {} samples across {} params is {}".format(N, num_vars, samples_per_param))
    
    samples_per_param = int(samples_per_param)
    
    # experiment = np.empty((N+1, num_vars))
    experiment = np.array(nominal * (N+1)).reshape(N + 1, num_factors)
    
    cols = np.arange(num_factors)
    
    # Samples represent runs with values at the extremes of each parameter boundary
    pos = 1
    for i in range(num_factors):
        
        min_val = bounds[i][0]
        max_val = bounds[i][1]
        
        p_samp = np.linspace(min_val, max_val, samples_per_param)
        
        subset = slice(pos, pos+samples_per_param)
        experiment[subset, i] = p_samp
        
        pos += samples_per_param
    # End for
    
    return experiment
# End traditional_oat_sample()

    
def oat_analyze(problem, inputs, results):
    """Analyze a 'traditional' OAT result.
    
    Parameters
    ==========
    * problem : dict, the problem definition following SALib specs
    * inputs : np.array, samples used to generate results. 
               The first row is expected to hold nominal values.
    * results : np.array, model run results obtained using the samples.
                The first row is expected to hold nominal results.          
    """
    X = inputs
    Y = results
    
    num_results, num_factors = X.shape
    nominal_values = problem.get('nominal')
    groupings = int((len(Y) - 1) / num_factors)

    # Set up result array (ignores one row as it holds the nominal results)
    si_res = np.full((groupings, num_factors), fill_value=np.nan)
    
    col = 0
    idx = 0
    for i in range(1, num_results):
        diff = (X[i, col] - X[0, col])
        if diff == 0.0:
            si_res[idx, col] = 0.0
        else:
            try:
                si_res[idx, col] = (Y[i] - Y[0]) / (X[i, col] - X[0, col])
            except IndexError:
                print(idx, col)
                raise
        # End if
        
        if i % groupings == 0:
            col += 1
            idx = 0
            continue
        # End if
        
        idx += 1
    # End for
    
    return si_res
# End oat_analyze()

In [5]:
oat_samples = traditional_oat_sample(oat_spec, 5000)

In [6]:
Y = ishigami_wrapper(oat_samples)

In [7]:
oat_si = oat_analyze(oat_spec, oat_samples, Y)

In [8]:
oat_si.mean(axis=0)

array([0.49921414, 1.75941067, 0.36142324, 0.        ])

In [9]:
%%time
extremity_samples = oat_extremity_sample(oat_spec)

Wall time: 0 ns


In [10]:
%time
Y = ishigami_wrapper(extremity_samples)

Wall time: 0 ns


In [11]:
extremity_samples

array([[ 1.        ,  1.        ,  1.        ,  1.        ],
       [-3.14159265,  1.        ,  1.        ,  1.        ],
       [ 3.14159265,  1.        ,  1.        ,  1.        ],
       [ 1.        , -3.14159265,  1.        ,  1.        ],
       [ 1.        ,  3.14159265,  1.        ,  1.        ],
       [ 1.        ,  1.        , -3.14159265,  1.        ],
       [ 1.        ,  1.        ,  3.14159265,  1.        ],
       [ 1.        ,  1.        ,  1.        , -3.14159265],
       [ 1.        ,  1.        ,  1.        ,  3.14159265]])

In [12]:
Y

array([ 5.88213201,  4.95651393,  4.95651393,  0.92561808,  0.92561808,
       13.99467729, 13.99467729,  5.88213201,  5.88213201])

In [13]:
%time
oat_analyze(oat_spec, extremity_samples, Y).mean(axis=0)

Wall time: 0 ns


array([-0.10435844, -0.55882018,  0.91464567,  0.        ])

In [22]:
%%timeit
oat_samples = traditional_oat_sample(oat_spec, 5000)

848 µs ± 10.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [23]:
%%timeit
Y = ishigami_wrapper(oat_samples)

8.19 ms ± 48.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
%%timeit
oat_si = oat_analyze(oat_spec, oat_samples, Y)

6.85 ms ± 28.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
%%timeit
oat_si.mean(axis=0)

26.7 µs ± 154 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
